In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import requests # The requests library 
import bs4 as bs # BeautifulSoup4 is a Python library 
from collections import Counter
import re

df = pd.read_csv('clean-airbnb.csv',sep='\t')
df.head()

,Unnamed: 0,Unnamed: 0.1,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,...,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,expire
0,0,0,NaN,https://www.airbnb.com/rooms/2105840,NaN,4/2/17,Room w/priv. bath Duboce Triangle 1,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,...,10.0,9.0,NaN,SAN FRANCISCO,strict,2,1.16,NaN,NaN,0
1,1,1,NaN,https://www.airbnb.com/rooms/5505643,NaN,4/2/17,Modern Mission-Dolores apartment,"Top floor apartment in a two unit building, bu...",Full open floor living room/dining room/kitche...,"Top floor apartment in a two unit building, bu...",...,10.0,10.0,NaN,SAN FRANCISCO,strict,1,2.51,NaN,NaN,0
2,2,2,NaN,https://www.airbnb.com/rooms/746079,NaN,4/2/17,Castro/Corbett Heights Condo,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,...,10.0,9.0,SAN FRANCISCO SHORT-TERM RENTAL REGISTRATION N...,SAN FRANCISCO,flexible,1,4.30,NaN,NaN,0
3,3,3,NaN,https://www.airbnb.com/rooms/1304068,NaN,4/2/17,Historic SF Residence on a park,"Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...",...,10.0,10.0,STR-0001041,SAN FRANCISCO,moderate,1,0.23,NaN,NaN,0
4,4,4,NaN,https://www.airbnb.com/rooms/3116428,NaN,4/2/17,Sunny/stylish/central/quiet SF 2BR,"Sunny, stylish top floor apartment in a recent...",The apartment is located on the top floor of a...,"Sunny, stylish top floor apartment in a recent...",...,10.0,10.0,STR-0000683,SAN FRANCISCO,moderate,1,0.56,NaN,NaN,0


In [3]:
df.columns 

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ID', 'Listing Url', 'Scrape ID',
       'Last Scraped', 'Name', 'Summary', 'Space', 'Description',
       'Experiences Offered', 'Neighborhood Overview', 'Notes', 'Transit',
       'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url',
       'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Name',
       'Host Since', 'Host Location', 'Host About', 'Host Response Time',
       'Host Response Rate', 'Host Acceptance Rate', 'Host Thumbnail Url',
       'Host Picture Url', 'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price',

In [4]:
# check whether there is null for all columns
df.isnull().sum()

Unnamed: 0                           0
Unnamed: 0.1                         0
ID                                2770
Listing Url                          0
Scrape ID                         2770
                                  ... 
Calculated host listings count       0
Reviews per Month                  159
Geolocation                       2770
Features                          2770
expire                               0
Length: 92, dtype: int64

# Drop NaN values 

In [5]:
# columns which have string can't replace NaN
df = df.dropna(subset=['Host Since','First Review','Host Location','Last Review','Bathrooms',
                       'Beds','Host Response Time','Host Neighbourhood'])

# Drop some columns (columns which have All NAN values and non-useful)

In [6]:
# drop some columns 
df = df.drop(['Unnamed: 0.1','Listing Url','ID','Last Scraped','Name','Host URL','Host Name','Unnamed: 0','Host Thumbnail Url','Scrape ID','Picture Url','Country',
              'City','Experiences Offered','Host ID','Host Response Rate','Host Acceptance Rate','Neighbourhood Group Cleansed','State',
              'Square Feet','Calendar last Scraped','License','Jurisdiction Names','Cleaning Fee',
              'Extra People','Host Listings Count','Availability 30','Availability 60','Availability 90',
              'Has Availability','Geolocation','Features','Longitude','Latitude','Weekly Price','Monthly Price',
              'Zipcode','expire','Security Deposit','Thumbnail Url','Medium Url','Host Picture Url','XL Picture Url',
              'Market','Host Total Listings Count','Neighbourhood','Country Code','Smart Location','Street','Host Neighbourhood'],axis=1)

In [7]:
df.isnull().sum()

Summary                           106
Space                             215
Description                         0
Neighborhood Overview             524
Notes                             651
Transit                           466
Access                            476
Interaction                       524
House Rules                       312
Host Since                          0
Host Location                       0
Host About                        363
Host Response Time                  0
Host Verifications                  0
Neighbourhood Cleansed              0
Property Type                       0
Room Type                           0
Accommodates                        0
Bathrooms                           0
Bedrooms                            0
Beds                                0
Bed Type                            0
Amenities                           0
Price                               0
Guests Included                     0
Minimum Nights                      0
Maximum Nigh

# Replace NaN values with average of each columns

In [8]:
df['Review Scores Rating'] =df['Review Scores Rating'].fillna(df['Review Scores Rating'].mean())
df['Review Scores Accuracy'] =df['Review Scores Accuracy'].fillna(df['Review Scores Accuracy'].mean())
df['Review Scores Cleanliness'] =df['Review Scores Cleanliness'].fillna(df['Review Scores Cleanliness'].mean())
df['Review Scores Checkin'] =df['Review Scores Checkin'].fillna(df['Review Scores Checkin'].mean())
df['Review Scores Communication'] =df['Review Scores Communication'].fillna(df['Review Scores Communication'].mean())
df['Review Scores Location'] =df['Review Scores Location'].fillna(df['Review Scores Location'].mean())
df['Review Scores Value'] =df['Review Scores Value'].fillna(df['Review Scores Value'].mean())

# Cleaning Columns 

### Host location

In [9]:
# check Host location 
pd.unique(df['Host Location'])

array(['San Francisco, California, United States', 'US',
       'Noe Valley / Bernal Heights / Mission',
       'California, United States',
       'Los Angeles, California, United States',
       'Mill Valley, California, United States',
       'Oakland, California, United States',
       'Irvine, California, United States',
       'Berkeley, California, United States',
       'Penn Valley, California, United States',
       'San Anselmo, California, United States',
       'Joshua Tree, California, United States',
       'Byron, Illinois, United States',
       'Portland, Oregon, United States',
       'Redwood City, California, United States',
       'New York, New York, United States', 'SAN FRANCISCO, California',
       'Denver, Colorado, United States',
       'Tahoe City, California, United States', 'United States',
       'Hillsborough, California, United States',
       'Palm Springs, California, United States', 'CA',
       'Salt Lake City, Utah, United States', 'Bali, Indones

Only San Francisco and Noe Valley can count as San Francisco city. So if host location is in San Francisco, we will return 1. otherwise 0. 

In [10]:
# fillna as random string and convert a string to a lowercase 
#df['Host Location'].fillna('no value', inplace = True)
df['Host Location'] = df['Host Location'].str.lower()

In [11]:
# replace 1: san franicsco 0: otherwise
df['Host Location'] = df['Host Location'].str.contains('|'.join(['san francisco','noe valley'])).astype(int)

In [12]:
df['Host Location'].head(5)

0    1
1    1
2    1
3    1
4    1
Name: Host Location, dtype: int32

### Maximum Nights

In [13]:
pd.unique(df['Maximum Nights'])

array([        30,         90,         16,       1125,         14,
               12,          7,         28,         15,         10,
               20,         29,        120,        180,         27,
               21,         31,        100,          5,        365,
                6,        186,         25,         99,         26,
              356,         22,        300,        730,        150,
               60,       1825,         35,          9,          4,
               33,        367,         13,        360,         19,
            10000,          3,         24,          8,        125,
              540,         89,        364,         88,        999,
       2147483647,       2922,        188,        220,         56,
              200,         45,        720,         40,         70,
               32,         93,        760,         96,         59,
               17,        400,        330,        350,          1,
               95,        160,        102,        270,        

In [14]:
# remove some outliers
df = df.loc[df['Maximum Nights']!=10000]
df = df.loc[df['Maximum Nights']!=999999]
df = df.loc[df['Maximum Nights']!=2147483647]

### First Reviews and Last Reviews 

In [15]:
datetime_object = datetime.strptime('11/12/16', '%m/%d/%y')
datetime_object

datetime.datetime(2016, 11, 12, 0, 0)

In [16]:
first_review = pd.to_datetime(df['First Review'])
last_review = pd.to_datetime(df['Last Review'])
df['first_dt'] = first_review
df['last_dt'] = last_review
df['first_YearMonth'] = df['first_dt'].map(lambda x: x.year + x.month/12)
df['last_YearMonth'] = df['last_dt'].map(lambda x: x.year + x.month/12)
df['today'] =np.ones(len(df))*(2018+11/12)
df['duration_rev'] = df['last_YearMonth']-df['first_YearMonth']
df['Time since last review'] = df['today'] - df['last_YearMonth']

In [17]:
# drop columns that were created to calculate
df = df.drop(['first_dt','last_dt','today', 'first_YearMonth', 'last_YearMonth','First Review','Last Review'], axis=1)

### Amenities 

In [18]:
# Total number of amenities per listing
df['Amenities'][0]

'{Internet,"Wireless Internet",Heating,Washer,Dryer,"Smoke detector","Carbon monoxide detector","Fire extinguisher",Essentials,"24-hour check-in",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50","Self Check-In",Lockbox,"Private entrance"}'

In [19]:
df['Amenities_list']=df['Amenities'].apply(lambda x:x[1:-1]).str.split(',')
df['Number of Amenities'] = df['Amenities_list'].map(lambda x: len(x))

# drop original amenities column
df = df.drop(['Amenities','Amenities_list'],axis=1)

### Host Since 

In [20]:
#datetime.strptime(date_str, '%Y-%m-%d').strftime("%Y-%m")

df['Host Since'] = pd.to_datetime(df["Host Since"])
df['Host Since'] = df['Host Since'].map(lambda x: x.year + x.month/12)
df['Host Since']
df['Host Duration'] = (11/12 + 2018) - df['Host Since'] 
#fractional years 
df["Host Duration"]
#dropping Host since series 
df = df.drop(labels = 'Host Since', axis =1 )

### Host VERIFICATIONS 

In [21]:
#Host VERIFICATIONS 
#transform by adding number of verifications (more is higher ranked )
df['Verification_list']=df['Host Verifications'].apply(lambda x:x[1:-1]).str.split(',')
df["Host Verifications"] = df["Verification_list"].map(lambda x: len(x))
df = df.drop(labels = 'Verification_list', axis =1 )

# Crime data

https://www.areavibes.com/san+francisco-ca/mission/livability/?ll=37.76009+-122.4145
The crimes will be “scored” depending on the crime category. SF will be zoned into multiple regions, with each zone scoredon safety depending on the average crime rate “score.”

In [22]:
df["Neighbourhood Cleansed"].value_counts().index

Index(['Mission', 'Western Addition', 'Bernal Heights', 'Castro/Upper Market',
       'Noe Valley', 'Downtown/Civic Center', 'Haight Ashbury', 'Potrero Hill',
       'Outer Sunset', 'Inner Richmond', 'South of Market', 'Excelsior',
       'Outer Mission', 'Marina', 'North Beach', 'Pacific Heights',
       'Outer Richmond', 'Russian Hill', 'West of Twin Peaks', 'Nob Hill',
       'Parkside', 'Inner Sunset', 'Bayview', 'Ocean View',
       'Financial District', 'Twin Peaks', 'Glen Park', 'Chinatown',
       'Visitacion Valley', 'Crocker Amazon', 'Diamond Heights', 'Seacliff',
       'Presidio Heights', 'Lakeshore', 'Golden Gate Park', 'Presidio'],
      dtype='object')

In [23]:
dic = ({"Neighbourhood Cleansed":['Mission', 'Western Addition', 'Bernal Heights', 'Castro/Upper Market',
       'Downtown/Civic Center', 'Noe Valley', 'Haight Ashbury', 'Outer Sunset',
       'Potrero Hill', 'South of Market', 'Inner Richmond', 'Outer Mission',
       'Excelsior', 'North Beach', 'Marina', 'Pacific Heights',
       'Outer Richmond', 'Russian Hill', 'Nob Hill', 'West of Twin Peaks',
       'Inner Sunset', 'Bayview', 'Parkside', 'Financial District',
       'Ocean View', 'Glen Park', 'Chinatown', 'Twin Peaks', 'Diamond Heights',
       'Visitacion Valley', 'Crocker Amazon', 'Presidio Heights', 'Lakeshore',
       'Seacliff', 'Golden Gate Park', 'Treasure Island/YBI', 'Presidio'], "crime": ["F", "F", "D-", "F", "F", "B+", "F", "A-", 
                                                                                    "F", "F", "B-", "F", "D+", "B-", "F", "D+",
                                                                                    "B", "F", "D", "C", "B+", "F", "C+", 
                                                                                     "B-", "B", "B-", "F", "C", "D-", "F", "B+",
                                                                                    "C+", "F", "B-", "F", "F", "C+"]}
       
      )

In [24]:
neighbor_crime = pd.DataFrame.from_dict(dic)

In [25]:
df = df.merge(neighbor_crime, on = 'Neighbourhood Cleansed')

In [26]:
df[['Neighbourhood Cleansed', 'crime']].head(5)

,Neighbourhood Cleansed,crime
0,Castro/Upper Market,F
1,Castro/Upper Market,F
2,Castro/Upper Market,F
3,Castro/Upper Market,F
4,Castro/Upper Market,F


In [27]:
df['crime'] = df['crime'].replace({'F': 0, 'D-': 1, 'D': 2, 'D+': 3, 'C-': 4, 'C': 5, 'C+': 6, 'B-': 7, 'B': 8, 'B+':9, 'A-': 10, 'A': 11 } )

In [28]:
df[df['crime'] == 6].head()

,Summary,Space,Description,Neighborhood Overview,Notes,Transit,Access,Interaction,House Rules,Host Location,...,Review Scores Location,Review Scores Value,Cancellation Policy,Calculated host listings count,Reviews per Month,duration_rev,Time since last review,Number of Amenities,Host Duration,crime
2130,"Located in the Outer Sunset, near Ocean Beach,...",Our home is a cozy house with two floors. The ...,"Located in the Outer Sunset, near Ocean Beach,...","-Golden Gate park, 48 Ave SF Beach, Zoo, Stern...",We are thrilled to have you stay with us. Just...,"Muni L, Muni 29, BART, Cal-train. Uber, Freewa...","- Your bedroom, small convenient private bathr...",Call/text Sabrina (PHONE NUMBER HIDDEN) or or ...,We love having guests and hope that you will t...,1,...,10.0,9.0,flexible,2,0.87,0.916667,1.750000,19,2.833333,6
2131,"Cozy and well furnished, this 1 bedroom in-law...",The bedroom is furnished with a new queen size...,"Cozy and well furnished, this 1 bedroom in-law...",NaN,San Francisco Short-Term Residential Rental Re...,"It is close to public transportation, only 1 b...","It is close to public transportation, only 1 b...",I'm always available to help.,I expect my guests to respect all neighbors.,1,...,9.0,9.0,strict,1,0.59,1.166667,1.666667,7,4.083333,6
2132,"Spacious, modern, private 2 bedroom flat with ...","Clean, spacious and private, well-lit and newl...","Spacious, modern, private 2 bedroom flat with ...","Our house is in a quiet neighborhood, across t...",Short-Term Residential Rental Certificate Numb...,Our house is within a 1-minute walk of the L M...,Guests will be given a security code to enter ...,Nejdeh or Angela will be communicating with you.,- Quiet time is between 10pm and 7am. - Than...,1,...,10.0,10.0,flexible,1,3.51,0.500000,1.666667,21,2.333333,6
2133,Our home is newly remodeled is conveniently lo...,Our comfortable home was just remodeled with ...,Our home is newly remodeled is conveniently lo...,"Parkside district is a safe, quiet neighborhoo...",NaN,Bus stop and muni train is a few blocks from t...,You will have access to the entire upstairs of...,Guests have access to all amenities in the hou...,NaN,0,...,9.0,10.0,moderate,1,0.77,0.333333,1.833333,20,2.250000,6
2134,"Stunning, brand new 1-bedroom apartment. Perfe...",Meticulously designed space with open floor pl...,"Stunning, brand new 1-bedroom apartment. Perfe...","Our house is in a quiet, safe neighborhood. St...",Transit -- There is plenty or inexpensive publ...,We're a 5-minute walk to San Francisco MUNI pu...,Separate private entrance,Happy to help,Please be mindful that we live above you and h...,1,...,9.0,9.0,strict,1,2.64,0.916667,1.666667,17,5.583333,6


In [29]:
#df = df.drop(labels = 'Neighbourhood Cleansed', axis =1 )

In [30]:
len(df)

2289

In [31]:
df.isnull().sum()

Summary                           106
Space                             215
Description                         0
Neighborhood Overview             524
Notes                             650
Transit                           466
Access                            476
Interaction                       524
House Rules                       312
Host Location                       0
Host About                        362
Host Response Time                  0
Host Verifications                  0
Neighbourhood Cleansed              0
Property Type                       0
Room Type                           0
Accommodates                        0
Bathrooms                           0
Bedrooms                            0
Beds                                0
Bed Type                            0
Price                               0
Guests Included                     0
Minimum Nights                      0
Maximum Nights                      0
Calendar Updated                    0
Availability

In [33]:
df.to_csv('final-airbnb.csv', sep='\t')